# 第8章 ディープラーニングを用いた因果探索

#### ・本章ではディープラーニングを用いた因果探索について解説、実装する .

## 8-1 因果探索とGAN(Generative Adversarial Networks)の関係

#### ・2020年において、因果推論の分野でもディープラーニングを利用した研究が進んでいる .　具体例として、

#### グラフニューラルネットワークを用いた因果探索
#### 深層強化学習を用いた因果探索
#### GAN(Generative Adversarial Networks)を用いた因果探索

#### などが挙げられる .

#### ・本章ではディープラーニングを用いた因果探索手法の中でも、"SAM(Structural Agnostic Model)"について解説、実装する .　SAMはGANを用いた因果探索手法である .

#### ・本章の内容はPyTorchを用いたディープラーニングとGANの実装経験がないと理解が難しい部分が多いらしいです .

#### ・本章で出てくる実装はSAM論文の著者らのGitHubのコードを一部参考、使用している .

![alt text](pict1.png)

## GAN(Generative Adversarial Networks)とは

#### ・GANは大量の画像データを学習に使用し、実際には存在していない架空の画像を生成する技術として有名である .

#### GANは生成器(Generator,以下G)と呼ばれるニューラルネットワークと、識別器(Discriminator,以下D)と呼ばれる2種類のニューラルネットワークから構成される .

#### ・画像を生成する際に使用するのは、「学習済みの生成器G」のみである .　学習済みの生成器Gに入力としてノイズを与えると、そのノイズの値に応じた架空の画像データが生成され、Gから出力される .

#### 例えば、入力ノイズの次元が20次元、出力される画像のサイズが縦横30ピクセルずつであれば、テンソルサイズ[20]のノイズ入力を入力として、テンソルサイズ[3,30,30]の出力を作り出す .　出力テンソルの最初の次元の3はRGBの各色チャンネルを示す .

#### ・学習済み生成器Gを作る際に、手書き数字の画像を学習データとして与えた場合、手書き数字の画像を出力するような生成器Gを構築できる .

#### また、GANの種類によっては入力にノイズだけでなく、条件(手書き数字画像であれば0や1など数字の種類を指定)を入力する場合がある .　この場合は条件に応じた任意の数字の画像を生成させることができる(Conditional GANと呼ぶ .) .

#### ・ここで重要な点は、"生成器Gはノイズを入力に人が数字と認識できる画像を生成してくれる"点である .　例えば、3×30×30の画像サイズにおいて、各値は0から255の256通りの値をとるため、生成できる画像パターンは膨大になる .　この膨大なパターンの中から、人が見たときに手書き数字画像と思えるパターンを出力してくれるのが生成器Gである .

#### このとき、生成される画像は基本的には学習に使用した画像には含まれていないパターンの画像となる .　学習に使用した画像を表示するのではなく、学習に使用した画像の特徴をもった、新たな画像を生成してくれる .

#### ・では、そのような生成器Gを構築するために生成器Gのニューラルネットワークをどのように学習させれば良いかが問題となる .　ここで識別器Dが登場する .

#### 生成器Gが生成した画像が数字に見えるかどうかを、いちいち人が判定することは大変で時間的にも困難なので、識別器Dが人に代わって、「生成器Gが生成した画像が数字に見えるかどうか」を判定する .　正確には、「数字に見えるかどうか」の判定は難しいので、「学習データセットにある画像か、それともGで生成された画像か？」を判定させる .

### 識別器Dが学習データセットにある画像かGで生成された画像かうまく区別が付かないようになれば、Gで生成される画像は学習データの特徴をもった、人が見ても手書き数字に見える画像だと判断できる .

#### ・この学習の際に、識別器Dは学習データセットの画像と生成画像の判定がうまくできない状態からスタートし、生成器Gもまるで砂嵐のような画像しか生成できない状態からスタートする .

#### そして、生成器Gは「識別器Dが学習データセットの画像と勘違いする画像を生成するように」、識別器Dは「学習データセットにある画像か、それとも生成された画像か判定できるように」、それぞれのニューラルネットワークを更新していく .

#### 識別器Dも初期状態からスタートする理由は、はじめから完璧にGの生成画像を見抜かれてしまうと、Gの学習がうまくいかないからである .　DとGを徐々に切磋琢磨させながら、学習を進める .

#### ・これらを踏まえると、最終的に学習済み生成器Gは人が見たときに、学習データセットにある画像のような画像を生成できるようになる .　これがGANの大まかなイメージである .

#### ・本章の因果探索の場合では、「観測したデータと同じような特徴のデータを生み出す生成器Gを学習させることができれば、この生成器Gから観測データが生まれるしくみを解き明かし、因果ダイアグラムを描くことができる .　これが因果探索にGANを用いるモチベーションとなる .

## 8-2 SAM(Structural Agnostic Model)の概要

#### ・本節では、GANを用いた因果探索手法であるSAM(Structural Agnostic Model)の、生成器Gおよび識別器Dの概要を、図を用いて解説する .

## 識別器Dのネットワーク構造